In [1]:
import theano
import theano.tensor as tt
import pymc3 as pm
import numpy as np

In [2]:
def transformation(X):
    n = X.shape[0]
    X_i = tt.tile(X[:, np.newaxis], [1, n])
    X_j = tt.tile(X[np.newaxis, :], [n, 1])
    Y_ij = tt.switch(X_i<=X_j, X_i, X_j)
    return tt.triu(Y_ij)

X = tt.vector()
Y_ij = transformation(X)
Y_ij.eval({X: np.random.randn(5)})

array([[ 0.23996979,  0.23996979, -0.97203886,  0.23996979,  0.23996979],
       [ 0.        ,  1.86201468, -0.97203886,  0.99536964,  0.53849045],
       [-0.        , -0.        , -0.97203886, -0.97203886, -0.97203886],
       [ 0.        ,  0.        , -0.        ,  0.99536964,  0.53849045],
       [ 0.        ,  0.        , -0.        ,  0.        ,  0.53849045]])

In [138]:
y_index = tt.imatrix()
X = tt.vector()

output, _ = theano.scan(fn=lambda idx, x: tt.min(x[idx[0]:idx[1]+1]),
                        sequences=y_index,
                        non_sequences=X)
Y_ij = pm.expand_packed_triangular(n, output, lower=False)


n = 5
idxs_array = np.asarray(np.triu_indices(n), dtype=np.int32).T
x_value = np.random.randn(n)
print(output.eval({y_index: idxs_array, X: x_value}))
print(Y_ij.eval({y_index: idxs_array, X: x_value}))

[0.77869196 0.25313944 0.25313944 0.10087892 0.10087892 0.25313944
 0.25313944 0.10087892 0.10087892 1.21431893 0.10087892 0.10087892
 0.10087892 0.10087892 0.10978973]
[[0.77869196 0.25313944 0.25313944 0.10087892 0.10087892]
 [0.         0.25313944 0.25313944 0.10087892 0.10087892]
 [0.         0.         1.21431893 0.10087892 0.10087892]
 [0.         0.         0.         0.10087892 0.10087892]
 [0.         0.         0.         0.         0.10978973]]
